<a href="https://colab.research.google.com/github/nandanjunior/My-Marine-Route-AI/blob/main/Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import json

# Upload .robot files
uploaded = files.upload()

# --- Helpers ---
def extract_test_case_section(content):
    lines = content.splitlines()
    test_case_lines = []
    inside_test_section = False
    for line in lines:
        if line.strip().startswith("*** Test Cases ***"):
            inside_test_section = True
            continue
        if inside_test_section:
            if line.strip().startswith("***"):
                break
            test_case_lines.append(line.rstrip())
    return test_case_lines

def extract_structured_test_cases(lines):
    test_cases = []
    current_case = None
    for line in lines:
        if not line.strip():
            continue
        if not line.startswith(" ") and not line.startswith("#"):
            if current_case:
                test_cases.append(current_case)
            current_case = {"name": line.strip(), "body": [], "doc": ""}
        elif current_case:
            current_case["body"].append(line.rstrip())
            if "[Documentation]" in line:
                doc_text = line.split("[Documentation]")[-1].strip()
                if doc_text.startswith("..."):
                    doc_text = doc_text[3:].strip()
                current_case["doc"] = doc_text
    if current_case:
        test_cases.append(current_case)
    return test_cases

def to_finetune_format(test_cases):
    dataset = []
    for case in test_cases:
        instruction = case["doc"] if case["doc"] else f"Generate a test case for: {case['name']}"
        output = f"*** Test Cases ***\n{case['name']}\n"
        for step in case["body"]:
            output += f"{step}\n"
        dataset.append({
            "instruction": instruction.strip(),
            "input": "",
            "output": output.strip()
        })
    return dataset

# --- Main Processing ---
all_test_cases = []

for filename in uploaded:
    content = uploaded[filename].decode("utf-8")
    test_case_lines = extract_test_case_section(content)
    structured = extract_structured_test_cases(test_case_lines)
    all_test_cases.extend(structured)

# Convert to instruction-output dataset
finetune_data = to_finetune_format(all_test_cases)

# Save as JSONL
output_file = "robot_framework_finetune_dataset.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for item in finetune_data:
        f.write(json.dumps(item) + "\n")

# Download
files.download(output_file)


Saving Full_Test.robot to Full_Test (1).robot
Saving safety_Full_Test.robot to safety_Full_Test (1).robot


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json

file = json.load(open("json_extraction_dataset_500.json", "r"))
print(file[1])

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.1/295.1 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 801.2 

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/llama-3-8b-instruct"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.11: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.6.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset, Dataset
import json

# 1. Load your Alpaca-style JSONL file
data = []
with open("robot_framework_finetune_dataset.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        example = json.loads(line)
        instruction = example["instruction"]
        input_text = example["input"]
        output = example["output"]

        if input_text.strip():
            prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
        else:
            prompt = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"

        data.append({"text": prompt})

# 2. Convert to HuggingFace Dataset format
dataset = Dataset.from_list(data)


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=20,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/688 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 688 | Num Epochs = 3 | Total steps = 516
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 167,772,160 of 8,000,000,000 (2.10% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,1.411100
50,0.521400
75,0.418700
100,0.349200
125,0.277400
150,0.272300
175,0.284900
200,0.150500
225,0.175300
250,0.164400


Step,Training Loss
25,1.411100
50,0.521400
75,0.418700
100,0.349200
125,0.277400
150,0.272300
175,0.284900
200,0.150500
225,0.175300
250,0.164400


In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt
messages = [
    {"role": "user", "content": "Check on general voltage reading"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Check on general voltage reading<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The general voltage reading of a battery can be checked using a multimeter. Here's how:

1. Set the multimeter to the voltage range (e.g. 12V, 20V, etc.)
2. Touch the positive lead to the positive terminal of the battery
3. Touch the negative lead to the negative terminal of the battery
4. Read the voltage value from the multimeter.

Note: Make sure to follow proper safety procedures when working with batteries and electrical systems.

Here's how to check general voltage reading of a lead acid battery:

* 12V Lead Acid Battery: 12.00V
* 14V Lead Acid Battery: 14.00V
* 20V Lead Acid Battery: 20.00V

Here's how to check general voltage reading of a lithium-ion battery:

* 12V Lithium-Ion Battery: 12.00V
* 14V Lithium-Ion Battery: 14.00V
* 20V Lithium-Ion Battery: 20.00V

Please note that these values are general and can vary based on various 

In [1]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

NameError: name 'model' is not defined

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)